In [1]:
%matplotlib inline
import os,random
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import theano as th
import theano.tensor as T
from keras.utils import np_utils
import keras.models as models
from keras.layers import Input,merge
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten,MaxoutDense
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Deconv2D, UpSampling2D
from keras.layers.recurrent import LSTM
from keras.regularizers import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.datasets import mnist
import matplotlib.pyplot as plt
import seaborn as sns
import cPickle, random, sys, keras
from keras.models import Model
from IPython import display
from keras.utils import np_utils
import os, sys
import glob
import cPickle as pkl
import numpy as np
import PIL.Image as Image
from skimage.transform import resize
import scipy.misc
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.utils.visualize_util import plot

K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [2]:
'''

img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print np.min(X_train), np.max(X_train)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


'''

"\n\nimg_rows, img_cols = 28, 28\n\n# the data, shuffled and split between train and test sets\n(X_train, y_train), (X_test, y_test) = mnist.load_data()\n\nX_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)\nX_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)\nX_train = X_train.astype('float32')\nX_test = X_test.astype('float32')\nX_train /= 255\nX_test /= 255\n\nprint np.min(X_train), np.max(X_train)\n\nprint('X_train shape:', X_train.shape)\nprint(X_train.shape[0], 'train samples')\nprint(X_test.shape[0], 'test samples')\n\n\n"

In [3]:
#data path for input and target folders

data_path_input="/Users/pulkit/Desktop/yo/" #training_pulkit/input
print data_path_input
imgs_input = glob.glob(data_path_input + "/*.jpeg")


data_path_target="/Users/pulkit/Desktop/Deep Learning/Project Code/inpainting/training_pulkit/target" #
print data_path_target
imgs_target = glob.glob(data_path_target + "/*.jpeg")

# select some images from the input and target dataset

batch_idx = 0
batch_size = 100

batch_imgs_input = imgs_input[batch_idx*batch_size:(batch_idx+1)*batch_size]

batch_imgs_target = imgs_target[batch_idx*batch_size:(batch_idx+1)*batch_size]

#dataset variables below

img_input_x = np.zeros((len(batch_imgs_input),64,64,3)) #batch_imgs_input

img_target_y = np.zeros((len(batch_imgs_target),32,32,3)) #batch_imgs_target


print "creating nd array for input data"
#loop over the input images
for i, img_path in enumerate(batch_imgs_input): #batch_imgs_input

    img1 = Image.open(img_path)
    if len(np.array(img1).shape) == 3:
        img_input_x[i,:,:,:] = np.array(img1)
    else:
    	img_input_x[i,:,:,0] = np.array(img1)
    	img_input_x[i,:,:,1] = np.array(img1)
    	img_input_x[i,:,:,2] = np.array(img1)


print(np.shape(img_input_x))

print "creating nd array for target data"
#loop over the target images
for i, img_path in enumerate(batch_imgs_target): #batch_imgs_target

    img2 = Image.open(img_path)
    if len(np.array(img2).shape) == 3:
    	img_target_y[i,:,:,:] = np.array(img2)
    else:
    	img_target_y[i,:,:,0] = np.array(img2)
    	img_target_y[i,:,:,1] = np.array(img2)
    	img_target_y[i,:,:,2] = np.array(img2)

print(np.shape(img_target_y))


/Users/pulkit/Desktop/yo/
/Users/pulkit/Desktop/Deep Learning/Project Code/inpainting/training_pulkit/target
creating nd array for input data
(100, 64, 64, 3)
creating nd array for target data
(100, 32, 32, 3)


In [4]:
#prepare the data for train

a = img_input_x.astype('float32') / 255.
b = img_target_y.astype('float32') / 255.


X_train = a
Y_train = b

print X_train.shape
print Y_train.shape



(100, 64, 64, 3)
(100, 32, 32, 3)


In [5]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [6]:
shp = X_train.shape[1:]
print shp

dropout_rate = 0.25
opt = Adam(lr=1e-3)
dopt = Adam(lr=1e-4)
nch = 200

# Build Generative model ...
nch = 200
g_input = Input(shape=[12288])
x = Dense(100)(g_input)
H = Dense(nch*16*16, init='glorot_normal')(g_input)
H = BatchNormalization(mode=2)(H)
H = Activation('relu')(H)
H = Reshape( [16, 16, nch] )(H)
H = UpSampling2D(size=(2, 2))(H)
H = Convolution2D(3, 3, nch/2, border_mode='same', init='glorot_uniform')(H)
H = BatchNormalization(mode=2)(H)
H = Activation('relu')(H)
H = Convolution2D(3, 3, nch/4, border_mode='same', init='glorot_uniform')(H)
H = BatchNormalization(mode=2)(H)
H = UpSampling2D(size=(2, 2))(H)
g_V = Activation('relu')(H)

'''
nch = 200
g_input = Input(shape=[100])
H = Dense(nch*14*14, init='glorot_normal')(g_input)
H = BatchNormalization(mode=2)(H)
H = Activation('relu')(H)
H = Reshape( [nch, 14, 14] )(H)
H = UpSampling2D(size=(2, 2))(H)
H = Convolution2D(nch/2, 3, 3, border_mode='same', init='glorot_uniform')(H)
H = BatchNormalization(mode=2)(H)
H = Activation('relu')(H)
H = Convolution2D(nch/4, 3, 3, border_mode='same', init='glorot_uniform')(H)
H = BatchNormalization(mode=2)(H)
H = Activation('relu')(H)
H = Convolution2D(1, 1, 1, border_mode='same', init='glorot_uniform')(H)
g_V = Activation('sigmoid')(H)
'''

generator = Model(g_input,g_V)
generator.compile(loss='binary_crossentropy', optimizer=opt)
generator.summary()


# Build Discriminative model ...

d_input = Input(shape=(64, 64, 3))

x = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(d_input)
x = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = Dropout(dropout_rate)(x)
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Flatten()(x)
x = Dense(256)(x)
x = LeakyReLU(0.2)(x)
d_V = Dense(2,activation='softmax')(x)

'''
d_input = Input(shape=shp)
H = Convolution2D(256, 5, 5, subsample=(2, 2), border_mode = 'same', activation='relu')(d_input)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
H = Convolution2D(512, 5, 5, subsample=(2, 2), border_mode = 'same', activation='relu')(H)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
H = Flatten()(H)
H = Dense(256)(H)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
d_V = Dense(2,activation='softmax')(H)
'''

discriminator = Model(d_input,d_V)
discriminator.compile(loss='categorical_crossentropy', optimizer=dopt)
discriminator.summary()

# Freeze weights in the discriminator for stacked training
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
make_trainable(discriminator, False)
# Build stacked GAN model
gan_input = Input(shape=[12288])
H = generator(gan_input)
gan_V = discriminator(H)
GAN = Model(gan_input, gan_V)
GAN.compile(loss='categorical_crossentropy', optimizer=opt)
GAN.summary()


(64, 64, 3)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 12288)         0                                            
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 51200)         629196800   input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 51200)         102400      dense_2[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 51200)         0           batchnormalization_1[0][0]       
_______________________________________________________________________________

In [7]:
'''
k = Input(shape=(64, 64, 3))

y = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(k)
y = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(y)
y = MaxPooling2D((2, 2), border_mode='same')(y)
y = Dropout(dropout_rate)(y)
y = Flatten()(y)
input_test = Dense(100)(y)

yo = Model(k,input_test)
yo.compile(loss='binary_crossentropy', optimizer=opt)

y_hat = yo.predict(X_train[1:5])
print y_hat
'''

"\nk = Input(shape=(64, 64, 3))\n\ny = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(k)\ny = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(y)\ny = MaxPooling2D((2, 2), border_mode='same')(y)\ny = Dropout(dropout_rate)(y)\ny = Flatten()(y)\ninput_test = Dense(100)(y)\n\nyo = Model(k,input_test)\nyo.compile(loss='binary_crossentropy', optimizer=opt)\n\ny_hat = yo.predict(X_train[1:5])\nprint y_hat\n"

In [8]:
def plot_loss(losses):
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.figure(figsize=(10,8))
        plt.plot(losses["d"], label='discriminitive loss')
        plt.plot(losses["g"], label='generative loss')
        plt.legend()
        plt.show()

In [ ]:
def plot_gen(n_ex=1,dim=(4,4), figsize=(10,10) ):
    noise = np.random.uniform(0,1,size=[n_ex,12288])

    #print np.transpose(y_hat[0]).shape
    
    #generated_images = generator.predict(y_hat[0].reshape((1,100)))
    generated_images = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        img = generated_images[i]
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
'''
ntrain = 5000
trainidx = random.sample(range(0,X_train.shape[0]), ntrain)
print len(trainidx)
XT = X_train[trainidx,:,:,:]
print len(XT)
print XT.shape
'''

# Pre-train the discriminator network ...
noise_gen = np.random.uniform(0,1,size=[X_train.shape[0],12288])
generated_images = generator.predict(noise_gen)
print generated_images.shape
X = np.concatenate((X_train, generated_images))
n = X_train.shape[0]
y = np.zeros([2*n,2])
y[:n,1] = 1
y[n:,0] = 1

make_trainable(discriminator,True)
discriminator.fit(X,y, nb_epoch=1, batch_size=500)
y_hat = discriminator.predict(X)

In [ ]:
y_hat_idx = np.argmax(y_hat,axis=1)
y_idx = np.argmax(y,axis=1)
diff = y_idx-y_hat_idx
n_tot = y.shape[0]
n_rig = (diff==0).sum()
acc = n_rig*100.0/n_tot
print "Accuracy: %0.02f pct (%d of %d) right"%(acc, n_rig, n_tot)

In [ ]:
# set up loss storage vector
losses = {"d":[], "g":[]}

In [ ]:
def train_for_n(nb_epoch=1, plt_frq=1,BATCH_SIZE=500):

    for e in range(nb_epoch):  
        
        # Make generative images
        image_batch = X_train[np.random.randint(0,X_train.shape[0],size=BATCH_SIZE),:,:,:]    
        noise_gen = np.random.uniform(0,1,size=[BATCH_SIZE,12288])
        generated_images = generator.predict(noise_gen)
        
        # Train discriminator on generated images
        X = np.concatenate((image_batch, generated_images))
        y = np.zeros([2*BATCH_SIZE,2])
        y[0:BATCH_SIZE,1] = 1
        y[BATCH_SIZE:,0] = 1
        
        make_trainable(discriminator,True)
        d_loss  = discriminator.train_on_batch(X,y)
        losses["d"].append(d_loss)
    
        # train Generator-Discriminator stack on input noise to non-generated output class
        noise_tr = np.random.uniform(0,1,size=[BATCH_SIZE,12288])
        y2 = np.zeros([BATCH_SIZE,2])
        y2[:,1] = 1
        
        make_trainable(discriminator,False)
        g_loss = GAN.train_on_batch(noise_tr, y2 )
        losses["g"].append(g_loss)
        
        # Updates plots
        if e%plt_frq==plt_frq-1:
            plot_loss(losses)
            plot_gen()
        

In [ ]:
K.set_value(opt.lr, 1e-5)
K.set_value(dopt.lr, 1e-6)
train_for_n(nb_epoch=1, plt_frq=1,BATCH_SIZE=50)

In [ ]:
plot_loss(losses)

In [ ]:
plot_gen(1,(4,4),(10,10))

In [ ]:
plot_gen(4,(4,4),(10,10))

In [ ]:
def plot_real(n_ex=4,dim=(4,4), figsize=(10,10) ):
    
    #idx = np.random.randint(0,X_train.shape[0],n_ex)
    generated_images = X_train[:4,:,:,:]
    print generated_images.shape

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        img = generated_images[i]
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()



In [ ]:
plot_real()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(100) 
print pca 

x_flat = X_train[1].reshape(12288)
print x_flat.shape
X_proj = pca.fit_transform(x_flat)
print X_proj
print X_proj.shape

use more training data